### Import

In [1]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

### Input 생성

In [3]:
sample = " if you want you"

### 데이터  가공

char단위로 중복제거 후 딕셔너리의 저장

In [4]:
# make dictionary
char_set = list(set(sample)) # set에 넣게 되면 중복이 제거 된다.
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

{'o': 0, 't': 1, 'n': 2, 'f': 3, ' ': 4, 'a': 5, 'i': 6, 'w': 7, 'u': 8, 'y': 9}


순서 테스트

In [5]:
#a = 'abcdef'

#char_set = list(set(a))
#char_dic = {c: i for i, c in enumerate(char_set)}
#print(char_dic)

In [6]:
#for i in char_dic:
#    print(i)

In [7]:
# hyper parameters
dic_size = len(char_dic) # input size
hidden_size = len(char_dic) # hidden state size
learning_rate = 0.1

Index로 변환 (one-hot)

In [8]:
# data setting
sample_idx = [char_dic[c] for c in sample]
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]]

sample_idx 확인

In [9]:
char_dic

{'o': 0,
 't': 1,
 'n': 2,
 'f': 3,
 ' ': 4,
 'a': 5,
 'i': 6,
 'w': 7,
 'u': 8,
 'y': 9}

In [10]:
sample_idx

[4, 6, 3, 4, 9, 0, 8, 4, 7, 5, 2, 1, 4, 9, 0, 8]

x_data, y_data 확인

In [11]:
x_data

[[4, 6, 3, 4, 9, 0, 8, 4, 7, 5, 2, 1, 4, 9, 0]]

In [12]:
y_data

[[6, 3, 4, 9, 0, 8, 4, 7, 5, 2, 1, 4, 9, 0, 8]]

만든 데이터 tensor로 변경

In [13]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

/var/folders/vc/qmhd_frs3436xpj2_4dshbtw0000gn/T/ipykernel_24795/1455435457.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1643987637853/work/torch/csrc/utils/tensor_new.cpp:201.)
  X = torch.FloatTensor(x_one_hot)


### 모델 선언

In [14]:
# declare RNN
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True) # batch dimention이 먼저 오게 변경

### Loss function & optimizer

In [15]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss() # loss function 
optimizer = optim.Adam(rnn.parameters(), learning_rate) # optimizer

In [16]:
# start training
for i in range(50):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  2.4795987606048584 prediction:  [[5 7 7 7 6 5 8 7 0 7 8 8 7 6 5]] true Y:  [[6, 3, 4, 9, 0, 8, 4, 7, 5, 2, 1, 4, 9, 0, 8]] prediction str:  awwwiauwowuuwia
1 loss:  2.1969003677368164 prediction:  [[8 0 1 1 0 8 0 1 1 1 1 1 1 0 8]] true Y:  [[6, 3, 4, 9, 0, 8, 4, 7, 5, 2, 1, 4, 9, 0, 8]] prediction str:  uottouottttttou
2 loss:  1.9633570909500122 prediction:  [[8 0 4 1 0 8 4 8 0 2 4 4 8 0 8]] true Y:  [[6, 3, 4, 9, 0, 8, 4, 7, 5, 2, 1, 4, 9, 0, 8]] prediction str:  uo tou uon  uou
3 loss:  1.7607686519622803 prediction:  [[8 0 4 9 0 8 4 8 3 2 4 4 9 0 8]] true Y:  [[6, 3, 4, 9, 0, 8, 4, 7, 5, 2, 1, 4, 9, 0, 8]] prediction str:  uo you ufn  you
4 loss:  1.5906271934509277 prediction:  [[9 0 4 9 0 8 4 7 3 2 4 4 9 0 8]] true Y:  [[6, 3, 4, 9, 0, 8, 4, 7, 5, 2, 1, 4, 9, 0, 8]] prediction str:  yo you wfn  you
5 loss:  1.4644935131072998 prediction:  [[9 3 4 9 0 8 4 7 3 2 4 4 9 0 8]] true Y:  [[6, 3, 4, 9, 0, 8, 4, 7, 5, 2, 1, 4, 9, 0, 8]] prediction str:  yf you wfn  you
6 loss:  1

reference
https://gomguard.tistory.com/145